In [ ]:
import torch
import numpy as np
import os
import regex as re
from collections import defaultdict
from multiprocessing import Pool
from typing import BinaryIO

In [ ]:
torch.manual_seed(341)
np.random.seed(341)

In [ ]:
print(ord('牛')); print(chr(29275))

29275
牛


In [ ]:
print(chr(0)); chr(0)

 


'\x00'

In [ ]:
"this is a test string" + chr(0) + "string"

'this is a test string\x00string'

In [ ]:
print("this is a test string" + chr(0) + "string")

this is a test string string


In [ ]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode('utf-8') # byte string

print(utf8_encoded)

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'


In [ ]:
print(type(utf8_encoded))

<class 'bytes'>


In [ ]:
"".join([chr(o) for o in list(utf8_encoded)])

'hello! ã\x81\x93ã\x82\x93ã\x81«ã\x81¡ã\x81¯!'

In [ ]:
print(len(test_string)); print(len(utf8_encoded)); utf8_encoded.decode('utf-8')

13
23


'hello! こんにちは!'

In [ ]:
def decode_utf8_bytes_to_str(bytestring: bytes) -> str:
  return bytestring.decode('utf-8')

In [ ]:
decode_utf8_bytes_to_str("hello! こんにちは!".encode('utf-8'))

'hello! こんにちは!'

In [ ]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

In [ ]:
re.findall(PAT, "some text that i'll pre-tokenize!")

['some', ' text', ' that', ' i', "'ll", ' pre', '-', 'tokenize', '!']

In [ ]:
test = "don't pre-tokenize this, okay?"
pre_tokens = re.findall(PAT, test)
byte_sequences = [list(token.encode('utf-8')) for token in pre_tokens]

In [ ]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
TOKENIZER = re.compile(PAT)
SPECIAL_TOKEN = "<|endoftext|>"

In [ ]:
def find_chunk_boundaries(file: BinaryIO, desired_num_chunks: int, split_special_token: bytes) -> list[int]:
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096
    for bi in range(1, len(chunk_boundaries) - 1):
        pos = chunk_boundaries[bi]
        file.seek(pos)
        while True:
            mini_chunk = file.read(mini_chunk_size)
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = pos + found_at
                break
            pos += mini_chunk_size
    return sorted(set(chunk_boundaries))

In [ ]:
import os
from typing import BinaryIO

# Function to find Find safe chunk boundaries
def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), (
        "Must represent special token as a bytestring"
    )

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))

In [ ]:
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt


--2025-08-08 06:18:20--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-valid.txt
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.124, 18.172.134.88, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/42/7f/427f7497b6c6596c18b46d5a72e61364fcad12aa433c60a0dbd4d344477b9d81/6874bae9a4c1a4e7edcf0e53b86c17817e9cf881fc75ff2368da457b80c0585d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyStoriesV2-GPT4-valid.txt%3B+filename%3D%22TinyStoriesV2-GPT4-valid.txt%22%3B&response-content-type=text%2Fplain&Expires=1754637500&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NDYzNzUwMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Mi83Zi80MjdmNzQ5N2I2YzY1OTZjMThiNDZkNWE3MmU2MTM2NGZjYWQxMmFhNDMzYzYwYTBkYmQ0ZDM0NDQ3N2I5ZDgxLzY4NzRiYWU5YTRjMWE0ZTdlZGNmMGU1

In [ ]:
# Worker function to process chunks
def process_chunk(args):
    filename, start, end = args
    counts = defaultdict(int)

    with open(filename, "rb") as f:
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")

    docs = re.split(re.escape(SPECIAL_TOKEN), chunk)
    for doc in docs:
        for match in TOKENIZER.finditer(doc):
            token = match.group()
            counts[token] += 1

    return counts

# reducer function
def merge_counts(dicts):
    merged = defaultdict(int)
    for d in dicts:
        for k, v in d.items():
            merged[k] += v
    return merged

In [ ]:
# parallel pre tokenization function
def run_pre_tokenization_parallel(filename: str, num_processes: int = 4):
    with open(filename, "rb") as f:
        boundaries = find_chunk_boundaries(f, num_processes, SPECIAL_TOKEN.encode("utf-8"))

    args = [(filename, s, e) for s, e in zip(boundaries[:-1], boundaries[1:])]

    with Pool(num_processes) as pool:
        partial_counts = pool.map(process_chunk, args)

    full_counts = merge_counts(partial_counts)
    return full_counts

In [ ]:
%%time
tokenizer = re.compile(PAT)

pre_tokens = []

# serial impl
with open("/content/TinyStoriesV2-GPT4-valid.txt", "rb") as f:
    boundaries = find_chunk_boundaries(f, desired_num_chunks=4, split_special_token=b"<|endoftext|>")

    for start, end in zip(boundaries[:-1], boundaries[1:]):
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")
        pre_tokens += tokenizer.findall(chunk)  # regex pre-tokenization

CPU times: user 1.69 s, sys: 112 ms, total: 1.8 s
Wall time: 1.81 s


In [ ]:
len(pre_tokens), len(boundaries)

(5501965, 5)

In [ ]:
%%time
pre_token_counts = run_pre_tokenization_parallel("/content/TinyStoriesV2-GPT4-valid.txt", num_processes=5)
# print(sorted(pre_token_counts.items(), key=lambda x: -x[1])[:10])  # Top 10 tokens

CPU times: user 72.2 ms, sys: 38.8 ms, total: 111 ms
Wall time: 3.02 s


In [ ]:
print(sorted(pre_token_counts.items(), key=lambda x: -x[1])[:10])  # Top 10 tokens

[('.', 421616), (',', 235432), (' the', 211031), (' and', 196057), (' a', 152161), ('\n', 151989), (' to', 150493), (' was', 108019), (' They', 52425), (' it', 51670)]


In [ ]:
for k in pre_token_counts:
  if k[0] in ['|', '<', '>', ':', ]:
    print(k)

:
:"


In [ ]:
len(pre_token_counts)

13111

byte level encoding + collecting pair frequencies.

In [ ]:
!python --version

Python 3.11.13


In [ ]:
counts = list(pre_token_counts.items())

In [ ]:
freqs: dict[tuple[tuple[int, ...]], int] = {}

for token, count in pre_token_counts.items():
  token_bytes = token.encode('utf-8')
  # print(list(token_bytes))
  token_seq = tuple((b,) for b in token_bytes) # changed step
  freqs[token_seq] = count


In [ ]:
# count byte pair sequences
def get_pair_counts(freqs: dict[tuple[tuple[int, ...]], int]) -> dict[tuple[int, int], int]:

  pair_counts: dict[tuple[int, int], int] = defaultdict(int)
  for sequence, count in freqs.items():
    for i in range(len(sequence) - 1):
      pair_counts[(sequence[i], sequence[i + 1])] += count

  return pair_counts

In [ ]:
assert all(isinstance(tok, tuple) for seq in freqs.keys() for tok in seq)

Greedy BPE merge

In [ ]:
merges = []
vocab = {}
num_merges = 10000

# init vocab
next_id = 0
for b in range(256):
  vocab[(b,)] = next_id
  next_id += 1;

vocab[(tuple((b,) for b in b"<|endoftext|>"))] = next_id
special_token_id = next_id
next_id += 1

In [ ]:
%%time
for _ in range(num_merges):
  # calculate pair counts
  pair_counts = get_pair_counts(freqs)

  if not pair_counts:
    break

  # get highest frequency pair
  max_pair = max(pair_counts.items(), key=lambda x: (x[1], x[0]))[0]

  print(max_pair)
  merged_token = max_pair[0] + max_pair[1]
  # assign new token id to the merged pair
  new_token = max_pair
  if merged_token in vocab:
      continue
  vocab[merged_token] = next_id
  next_id += 1
  merges.append(merged_token)

  # vocab[(new_token)] = next_id
  # next_id += 1
  # merges.append(new_token)

  # replace max pair in all bytes (merging step)
  new_freqs = {}
  for seq, freq in freqs.items():
    new_seq = []
    i = 0
    while i < len(seq):
      if i < len(seq) - 1 and (seq[i], seq[i + 1]) == max_pair:
        new_seq.append(merged_token)
        i += 2 # jump ahead 2 bytes (merging step)
      else:
        new_seq.append(seq[i])
        i += 1

    new_freqs[tuple(new_seq)] = freq

  freqs = new_freqs

Streaming output truncated to the last 5000 lines.
((32, 83, 116, 101), (118, 101))
((32, 83), (117, 110, 100, 97, 121))
((32, 80), (97, 116))
((32, 72), (111, 112, 101))
((32, 67, 108), (97, 114, 97))
((121, 116, 104), (109,))
((105, 99, 107), (121,))
((105, 99), (105, 110, 103))
((100, 100), (121,))
((100,), (105, 110, 103))
((66,), (105, 114, 100))
((32, 117, 110, 105, 118, 101, 114), (115, 101))
((32, 115, 116, 114, 97), (105, 103, 104, 116))
((32, 115, 110, 111, 119), (98, 97, 108, 108))
((32, 115, 101, 101), (105, 110, 103))
((32, 114, 97, 110), (103,))
((32, 112, 111, 116, 97, 116, 111), (101, 115))
((32, 105, 115), (110,))
((32, 104, 105, 116), (115,))
((32, 101, 110, 99, 111, 117), (114, 97, 103, 101, 100))
((32, 99, 114, 101, 97, 116, 117, 114, 101), (115,))
((32, 99, 111, 108, 108, 101, 99, 116), (105, 111, 110))
((32, 98, 114, 101), (97, 107, 105, 110, 103))
((32, 98, 97, 99, 107), (112, 97, 99, 107))
((32, 98, 97), (107, 101, 100))
((32, 82), (101, 100))
((32, 70, 111), (1

In [ ]:
len(merges), len(vocab)

(10000, 10257)

In [ ]:
merges[:5]

[(32, 116), (104, 101), (32, 97), (32, 115), (32, 119)]

BPE encoding stage (tokenizer)

In [ ]:
nt = "hello"
byte_seq = [(b,) for b in nt.encode('utf-8')]  # list of byte tuples
merge_set = set(merges)

In [ ]:
while True:
  merged = False
  i = 0
  new_seq = []
  while i < len(byte_seq):
    if i < len(byte_seq) - 1 and (byte_seq[i], byte_seq[i + 1]) in merge_set:
      new_seq.append(byte_seq[i] + byte_seq[i + 1])
      i += 2
      merged = True
    else:
      new_seq.append(byte_seq[i])
      i += 1

  byte_seq = new_seq
  if not merged:
    break

In [ ]:
byte_seq

[(104,), (101,), (108,), (108,), (111,)]

In [ ]:
def bpe_encode(text: str, merges: list[tuple], vocab: dict) -> list[int]:
    # Step 1: Pre-tokenize
    pre_tokens = re.findall(PAT, text)

    token_ids = []
    merge_set = set(merges)

    for token in pre_tokens:
        byte_seq = [(b,) for b in token.encode('utf-8')]

        # Apply BPE merge logic
        while True:
            merged = False
            i = 0
            new_seq = []
            while i < len(byte_seq):
                if i < len(byte_seq) - 1 and (byte_seq[i], byte_seq[i+1]) in merge_set:
                    new_seq.append(byte_seq[i] + byte_seq[i+1])
                    i += 2
                    merged = True
                else:
                    new_seq.append(byte_seq[i])
                    i += 1
            byte_seq = new_seq
            if not merged:
                break

        # Convert to token IDs
        for tok in byte_seq:
            token_ids.append(vocab[tok])

    return token_ids

In [ ]:
class BPETokenizer():

  def __init__(self, vocab_size: int, specical_tokens: list[str] = None):
    pass

  def train(self, input_path:str):
    """Runs pre-tokenization, BPE Merging, and builds vocab + merges."""
    pass

  def get_vocab(self):
    pass



In [ ]:
from typing import IO, Any, BinaryIO
from collections.abc import Iterable
import os
import regex as re
from typing import BinaryIO
from collections import defaultdict
# from multiprocessing import Pool
from multiprocessing import get_context

import numpy.typing as npt
import torch
from torch import Tensor
SPECIAL_TOKEN = "<|endoftext|>"
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
TOKENIZER = re.compile(PAT)

In [ ]:
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt

--2025-08-11 07:03:53--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStoriesV2-GPT4-train.txt
Resolving huggingface.co (huggingface.co)... 3.169.137.119, 3.169.137.5, 3.169.137.19, ...
Connecting to huggingface.co (huggingface.co)|3.169.137.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/42/7f/427f7497b6c6596c18b46d5a72e61364fcad12aa433c60a0dbd4d344477b9d81/6418d412de72888f52b5142c761ac21a582f7d1166f0bfbdb5f03ccfdec90443?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyStoriesV2-GPT4-train.txt%3B+filename%3D%22TinyStoriesV2-GPT4-train.txt%22%3B&response-content-type=text%2Fplain&Expires=1754899433&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NDg5OTQzM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy80Mi83Zi80MjdmNzQ5N2I2YzY1OTZjMThiNDZkNWE3MmU2MTM2NGZjYWQxMmFhNDMzYzYwYTBkYmQ0ZDM0NDQ3N2I5ZDgxLzY0MThkNDEyZGU3Mjg4OGY1MmI1MTQyYzc2

In [ ]:
!wget https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
!gunzip owt_train.txt.gz

--2025-08-09 11:42:17--  https://huggingface.co/datasets/stanford-cs336/owt-sample/resolve/main/owt_train.txt.gz
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.90, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/56/35/56359101d2f3062d0e2aef6c02f65e54f35b2c4e53eaf92f8116bf2877b7df2f/b19ae88cfbc4016b304c348522455fe38ebac48fffed955adcc7191a89e38ccf?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27owt_train.txt.gz%3B+filename%3D%22owt_train.txt.gz%22%3B&response-content-type=application%2Fgzip&Expires=1754743337&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1NDc0MzMzN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzU2LzM1LzU2MzU5MTAxZDJmMzA2MmQwZTJhZWY2YzAyZjY1ZTU0ZjM1YjJjNGU1M2VhZjkyZjgxMTZiZjI4NzdiN2RmMmYvYjE5YWU4OGNmYmM0MDE2YjMwNGMzNDg1MjI0NTVmZTM4ZWJhYzQ

In [ ]:
# Function to find Find safe chunk boundaries
def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), (
        "Must represent special token as a bytestring"
    )

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))


# helper BPE train functions. (to run stuff in parallel)
def process_chunk(args):
    filename, start, end = args
    counts = defaultdict(int)

    with open(filename, "rb") as f:
        f.seek(start)
        chunk = f.read(end - start).decode("utf-8", errors="ignore")

    docs = re.split(re.escape(SPECIAL_TOKEN), chunk)
    for doc in docs:
        for match in TOKENIZER.finditer(doc):
            token = match.group()
            counts[token] += 1

    return counts

def merge_counts(dicts):
    merged = defaultdict(int)
    # current_merge = defaultdict(int)

    # for current_token in current_merge:
    #     for k, v as (p, q):
    #         with torch.nograd():
    #             current_grad += LR_scheduler * math.cos(1 + )
    for d in dicts:
        for k, v in d.items():
            merged[k] += v
    return merged

def run_pre_tokenization_parallel(filename: str, num_processes: int = 4):
    with open(filename, "rb") as f:
        boundaries = find_chunk_boundaries(f, num_processes, SPECIAL_TOKEN.encode("utf-8"))

    args = [(filename, s, e) for s, e in zip(boundaries[:-1], boundaries[1:])]

    # with Pool(num_processes) as pool:
    with get_context("fork").Pool(num_processes) as pool:
        partial_counts = pool.map(process_chunk, args)

    full_counts = merge_counts(partial_counts)
    return full_counts

# count byte pair sequences
def get_pair_counts(freqs: dict[tuple[tuple[int, ...]], int]) -> dict[tuple[int, int], int]:

  pair_counts: dict[tuple[int, int], int] = defaultdict(int)
  for sequence, count in freqs.items():
    for i in range(len(sequence) - 1):
      pair_counts[(sequence[i], sequence[i + 1])] += count

  return pair_counts


def run_train_bpe(
    input_path: str | os.PathLike,
    vocab_size: int,
    special_tokens: list[str],
    **kwargs,
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    """Given the path to an input corpus, run train a BPE tokenizer and
    output its vocabulary and merges.

    Args:
        input_path (str | os.PathLike): Path to BPE tokenizer training data.
        vocab_size (int): Total number of items in the tokenizer's vocabulary (including special tokens).
        special_tokens (list[str]): A list of string special tokens to be added to the tokenizer vocabulary.
            These strings will never be split into multiple tokens, and will always be
            kept as a single token. If these special tokens occur in the `input_path`,
            they are treated as any other string.

    Returns:
        tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
            vocab:
                The trained tokenizer vocabulary, a mapping from int (token ID in the vocabulary)
                to bytes (token bytes)
            merges:
                BPE merges. Each list item is a tuple of bytes (<token1>, <token2>),
                representing that <token1> was merged with <token2>.
                Merges are ordered by order of creation.
    """

    pre_token_counts = run_pre_tokenization_parallel(input_path, num_processes=4)
    merges = []
    vocab = {}
    # num_merges = vocab_size
    num_merges = vocab_size - len(special_tokens) - 256

    # init the vocab
    next_id = 0
    for b in range(256):
        # vocab[(b,)] = next_id
        # vocab[next_id] = (b,)
        vocab[next_id] = bytes([b])
        next_id += 1

    # Add special token(s)
    for tok in special_tokens:
        vocab[next_id] = tok.encode("utf-8")
        next_id += 1

    # vocab[(tuple((b,) for b in b"<|endoftext|>"))] = next_id
    # vocab[tuple(b"<|endoftext|>")] = next_id
    # special_token_id = next_id
    # next_id += 1

    freqs: dict[tuple[tuple[int, ...]], int] = {}

    for token, count in pre_token_counts.items():
        token_bytes = token.encode('utf-8')
        # print(list(token_bytes))
        token_seq = tuple((b,) for b in token_bytes) # changed step
        freqs[token_seq] = count

    for _ in range(num_merges):
        pair_counts = get_pair_counts(freqs)

        if not pair_counts:
            break

        # get the highest pair freq to merge.
        max_pair = max(pair_counts.items(), key= lambda x: (x[1], x[0]))[0]
        merged_token = max_pair[0] + max_pair[1]

        merges.append( (bytes(max_pair[0]), bytes(max_pair[1])) )
        if merged_token in vocab:
            continue

        # vocab[merged_token] = next_id
        vocab[next_id] = bytes(merged_token)
        next_id += 1
        # merges.append(merged_token)


        new_freqs = {}

        for seq, freq in freqs.items():
            new_seq = []
            i = 0
            while i < len(seq):
                if i < len(seq) - 1 and (seq[i], seq[i + 1]) == max_pair:
                    new_seq.append(merged_token)
                    i += 2 # jump ahead 2 bytes (merging step)
                else:
                    new_seq.append(seq[i])
                    i += 1

            new_freqs[tuple(new_seq)] = freq

        freqs = new_freqs

    # to return vocabulary and merges.
    return vocab, merges
    # raise NotImplementedError

In [ ]:
%%time
vocab, merges = run_train_bpe(input_path="/content/TinyStoriesV2-GPT4-train.txt", vocab_size=10000, special_tokens=["<|endoftext|>"])

CPU times: user 33min 38s, sys: 1.31 s, total: 33min 40s
Wall time: 42min 48s


In [ ]:
merges_copy = merges.copy()
vocab_copy = vocab.copy()
# !cat head /content/TinyStoriesV2-GPT4-train.txt

In [ ]:
merges

[(b' ', b't'),
 (b'h', b'e'),
 (b' ', b'a'),
 (b' ', b's'),
 (b' ', b'w'),
 (b'n', b'd'),
 (b' t', b'he'),
 (b'e', b'd'),
 (b' ', b'b'),
 (b' t', b'o'),
 (b' a', b'nd'),
 (b' ', b'h'),
 (b' ', b'f'),
 (b'i', b'n'),
 (b' ', b'T'),
 (b' w', b'a'),
 (b'r', b'e'),
 (b'i', b't'),
 (b'o', b'u'),
 (b' ', b'l'),
 (b' ', b'd'),
 (b' ', b'c'),
 (b' ', b'p'),
 (b'a', b'y'),
 (b' ', b'm'),
 (b'e', b'r'),
 (b' wa', b's'),
 (b'o', b'm'),
 (b' T', b'he'),
 (b' ', b'he'),
 (b'i', b's'),
 (b'a', b'r'),
 (b' ', b'n'),
 (b'i', b'm'),
 (b'o', b'n'),
 (b' s', b'a'),
 (b'i', b'd'),
 (b'l', b'l'),
 (b' h', b'a'),
 (b' ', b'g'),
 (b'a', b't'),
 (b' ', b'S'),
 (b'in', b'g'),
 (b'o', b't'),
 (b'e', b'n'),
 (b'a', b'n'),
 (b'l', b'e'),
 (b'o', b'r'),
 (b'i', b'r'),
 (b'a', b'm'),
 (b'e', b't'),
 (b' ', b'H'),
 (b' ', b'it'),
 (b' t', b'h'),
 (b'i', b'g'),
 (b' The', b'y'),
 (b' ', b'in'),
 (b'i', b'l'),
 (b' p', b'l'),
 (b' H', b'e'),
 (b' ', b'"'),
 (b'o', b'w'),
 (b'v', b'er'),
 (b'r', b'i'),
 (b'u', b't'),
 (

In [ ]:
len(vocab_copy), len(merges_copy)

(10000, 9743)

In [ ]:
import json

merges = merges_copy
vocab = vocab_copy
# save vocab
with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump({k: v.decode("utf-8", errors="replace") for k, v in vocab.items()}, f, ensure_ascii=False)

# save merges
# with open("merges.txt", "w", encoding="utf-8") as f:
#     for tok1, tok2 in merges:
#         f.write(f"{tok1.decode('utf-8', errors='replace')} {tok2.decode('utf-8', errors='replace')}\n")

# with open("merges.txt", "w", encoding="utf-8") as f:
#     for tok1, tok2 in merges:
#         f.write(f"{tok1.decode('utf-8')} {tok2.decode('utf-8')}\n")

# with open("merges.txt", "w", encoding="utf-8") as f:
#     for tok1, tok2 in merges:
#         f.write(f"{tok1.decode('utf-8', errors='ignore')} {tok2.decode('utf-8', errors='ignore')}\n")

with open("merges.txt", "w", encoding="utf-8") as f:
    for tok1, tok2 in merges:
        f.write(f"{tok1!r} {tok2!r}\n")  # Writes them as Python byte literals

In [ ]:
# Load vocab
with open("vocab.json", "r", encoding="utf-8") as f:
    vocab_str = json.load(f)
vocab = {int(k): v.encode("utf-8") for k, v in vocab_str.items()}

# Load merges
merges = []
with open("merges.txt", "r", encoding="utf-8") as f:
    for line in f:
        tok1, tok2 = line.rstrip("\n").split(" ", 1)
        merges.append((tok1.encode("utf-8"), tok2.encode("utf-8")))

In [ ]:
len(vocab), len(merges)

(10000, 9743)

In [ ]:
type(vocab)

dict

In [ ]:
byte_to_id = {(v): k for k, v in vocab.items()}
vocab

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [ ]:
def bpe_encode(text: str, merges: list[tuple[bytes, bytes]], vocab: dict, tokenizer) -> list[int]:
    token_ids = []

    # Step 0 — Make token->id map for quick lookup
    byte_to_id = {v: k for k, v in vocab.items()}

    # Pre-tokenize
    pre_tokens = tokenizer.findall(text)

    # Turn merges into a dict for fast lookup of merge rank
    merge_ranks = {pair: i for i, pair in enumerate(merges)}

    for token in pre_tokens:
        byte_seq = [(b,) for b in token.encode('utf-8')]

        while True:
            pairs = [(byte_seq[i], byte_seq[i+1]) for i in range(len(byte_seq)-1)]
            if not pairs:
                break

            # Find the highest priority merge (lowest rank)
            ranked_pairs = [(merge_ranks[p], p) for p in pairs if p in merge_ranks]
            if not ranked_pairs:
                break

            _, best_pair = min(ranked_pairs, key=lambda x: x[0])

            # Merge occurrences of this pair
            new_seq = []
            i = 0
            while i < len(byte_seq):
                if i < len(byte_seq)-1 and (byte_seq[i], byte_seq[i+1]) == best_pair:
                    new_seq.append(byte_seq[i] + byte_seq[i+1])
                    i += 2
                else:
                    new_seq.append(byte_seq[i])
                    i += 1
            byte_seq = new_seq

        # Map to IDs
        for b in byte_seq:
          print(b)
          token_ids.append(byte_to_id[b])

    return token_ids


In [ ]:
import regex as re

def bpe_encode(text: str, merges: list[tuple[bytes, bytes]], vocab: dict[bytes, int], TOKENIZER) -> list[int]:
    """
    Encode text using trained byte-level BPE merges.

    Args:
        text: Input text string.
        merges: List of merges (tuples of bytes, bytes) from training in order of creation.
        vocab: Mapping from bytes token -> token ID.
        pattern: Regex pattern for pre-tokenization (same as training).

    Returns:
        List of token IDs.
    """

    # Step 0 — Make token->id map for quick lookup
    byte_to_id = {v: k for k, v in vocab.items()}

    # Step 1: Pre-tokenization
    pre_tokens = TOKENIZER.findall(text)

    # For faster lookup: map merge tuple to its order
    merge_ranks = {pair: i for i, pair in enumerate(merges)}

    token_ids = []

    for token in pre_tokens:
        # Step 2: Represent each pre-token as a list of single-byte tuples
        byte_seq = [(b,) for b in token.encode('utf-8')]
        byte_seq = [bytes(b) for b in byte_seq]
        # Step 3: Apply merges in training order
        while True:
            # Find all adjacent pairs
            pairs = [(byte_seq[i], byte_seq[i + 1]) for i in range(len(byte_seq) - 1)]
            if not pairs:
                break

            # Find best pair (lowest rank = earliest merge in training)
            ranked_pairs = [(merge_ranks[p], p) for p in pairs if p in merge_ranks]
            if not ranked_pairs:
                break

            _, best_pair = min(ranked_pairs)

            # Merge the best pair throughout sequence
            new_seq = []
            i = 0
            while i < len(byte_seq):
                if i < len(byte_seq) - 1 and (byte_seq[i], byte_seq[i + 1]) == best_pair:
                    new_seq.append(byte_seq[i] + byte_seq[i + 1])
                    i += 2
                else:
                    new_seq.append(byte_seq[i])
                    i += 1
            byte_seq = new_seq

        # Step 4: Convert each final symbol to its vocab ID
        # for tok in byte_seq:
        #     token_ids.append(vocab[bytes(tok)])  # bytes(tok) matches vocab keys
        for b in byte_seq:
          print(byte_to_id[b])
          token_ids.append(byte_to_id[b])

    return token_ids


In [ ]:
# vocab[bytes(' k')]

In [ ]:
bpe_encode("the cat sat on the mat", merges, vocab, TOKENIZER)

116
104
101
32
99
97
116
32
115
97
116
32
111
110
32
116
104
101
32
109
97
116


[116,
 104,
 101,
 32,
 99,
 97,
 116,
 32,
 115,
 97,
 116,
 32,
 111,
 110,
 32,
 116,
 104,
 101,
 32,
 109,
 97,
 116]

In [ ]:
# vocab[t for t in bpe_encode("the cat sat on the mat", merges, vocab, TOKENIZER)]
# vocab[2492]
bpe_encode("this is the one we are looking for", merges, vocab, TOKENIZER)

116
104
105
115
32
105
115
32
116
104
101
32
111
110
101
32
119
101
32
97
114
101
32
108
111
111
107
105
110
103
32
102
111
114


[116,
 104,
 105,
 115,
 32,
 105,
 115,
 32,
 116,
 104,
 101,
 32,
 111,
 110,
 101,
 32,
 119,
 101,
 32,
 97,
 114,
 101,
 32,
 108,
 111,
 111,
 107,
 105,
 110,
 103,
 32,
 102,
 111,
 114]

In [ ]:
s = "this is the one we are looking for"

pre = TOKENIZER.findall(s)

# [list(b.encode('utf-8')) for b in pre]
merge_ranks = {pair: i for i, pair in enumerate(merges)}
byte_to_id = {v: k for k, v in vocab.items()}

tok_ids = []
print(merge_ranks)
for token in pre:

  byte_seq = [(b,) for b in token.encode('utf-8')]
  byte_seq = [bytes(b) for b in byte_seq]

  pairs = [(byte_seq[i], byte_seq[i + 1]) for i in range(len(byte_seq) - 1)]
  print(pairs)
  if not pairs:
    break
  # print(byte_seq)

  # iterate in merges and rank the pairs
  ranks = [(merge_ranks[pair], pair) for pair in pairs if pair in merge_ranks]
  if not ranks:
    break
  _, min_pair = min(ranks) # don't need the token id

  # merge this pair across all the pairs in the sequence
  i = 0
  new_seq = []
  while i < len(byte_seq):

    if i + 1 < len(byte_seq) and (byte_seq[i], byte_seq[i + 1]) == min_pair:
      new_seq.append(byte_seq[i] + byte_seq[i + 1])
      i += 2 # skip two bytes

    else:
      new_seq.append(byte_seq[i])
      i += 1
  byte_seq = new_seq

  for b in byte_seq:
    tok_ids.append(byte_to_id[b])

[vocab[t] for t in tok_ids]

{(b"b'", b"' b't'"): 0, (b"b'h'", b"b'e'"): 1, (b"b'", b"' b'a'"): 2, (b"b'", b"' b's'"): 3, (b"b'", b"' b'w'"): 4, (b"b'n'", b"b'd'"): 5, (b"b'", b"t' b'he'"): 6, (b"b'e'", b"b'd'"): 7, (b"b'", b"' b'b'"): 8, (b"b'", b"t' b'o'"): 9, (b"b'", b"a' b'nd'"): 10, (b"b'", b"' b'h'"): 11, (b"b'", b"' b'f'"): 12, (b"b'i'", b"b'n'"): 13, (b"b'", b"' b'T'"): 14, (b"b'", b"w' b'a'"): 15, (b"b'r'", b"b'e'"): 16, (b"b'i'", b"b't'"): 17, (b"b'o'", b"b'u'"): 18, (b"b'", b"' b'l'"): 19, (b"b'", b"' b'd'"): 20, (b"b'", b"' b'c'"): 21, (b"b'", b"' b'p'"): 22, (b"b'a'", b"b'y'"): 23, (b"b'", b"' b'm'"): 24, (b"b'e'", b"b'r'"): 25, (b"b'", b"wa' b's'"): 26, (b"b'o'", b"b'm'"): 27, (b"b'", b"T' b'he'"): 28, (b"b'", b"' b'he'"): 29, (b"b'i'", b"b's'"): 30, (b"b'a'", b"b'r'"): 31, (b"b'", b"' b'n'"): 32, (b"b'i'", b"b'm'"): 33, (b"b'o'", b"b'n'"): 34, (b"b'", b"s' b'a'"): 35, (b"b'i'", b"b'd'"): 36, (b"b'l'", b"b'l'"): 37, (b"b'", b"h' b'a'"): 38, (b"b'", b"' b'g'"): 39, (b"b'a'", b"b't'"): 40, (b"b'", b"' 

[]

In [ ]:
s = "this is the one we are looking for"
"""
["we have text"] -> pre-tokenize -> ["we", "have", "text"] ->

                                    (b'w', b'e') ->

                                    loop through merges (iteratively keep merging after finding them in the merge set)
"""

pre = TOKENIZER.findall(s)
merge_set = set(merges)
for token in pre:
  byte_seq = [(b,) for b in token.encode('utf-8')]

  byte_seq = [bytes(b) for b in byte_seq]
  # print(byte_seq)

  # iterate over the merge patterns and apply merge over the sequences

  while True:
    i = 0
    new_seq = []
    while i < len(byte_seq):

      if i + 1 < len(byte_seq) and (byte_seq[i], byte_seq[i + 1]) in merge_set:
        new_seq.append(byte_seq[i] + byte_seq[i + 1])
        i += 2
        # pass
      else:
        new_seq.append(byte_seq[i])
        i += 1


KeyboardInterrupt: 

True

In [ ]:
from collections.abc import Iterable, Iterator


In [ ]:
class Tokenizer():
  def __init__(self, vocab: dict[int, bytes], merges: list[tuple[bytes, bytes]], special_tokens: list[str] = None):
    self.vocab = vocab
    self.merges = merges
    self.special_tokens = special_tokens
    pass

  def from_file(cls, vocab_file_path: str, merges_filepath: str, special_tokens: list[str] = None):
    pass

  def encode(self, text: str) -> list[int]:
    pass

  def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
    pass

  def decode(self, ids: list[int]) -> str:
    pass

In [ ]:
class Transformer():

  def __init__(self):
    pass